In [2]:
import torch 
import numpy as np
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable


# Hyper Parameters
num_epochs = 5
batch_size = 100
learning_rate = 0.001

# MNIST Dataset
train_dataset = dsets.MNIST(root='./data/',
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./data/',
                           train=False, 
                           transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

Processing...
Done!


In [3]:
# CNN Model (2 conv layer)
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, padding=2),
            #nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.dropout1 = nn.Dropout(0.5)
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, padding=2),
            #nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.dropout2 = nn.Dropout(0.5)
        self.fc = nn.Linear(7*7*32, 10)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.dropout1(out)
        out = self.layer2(out)
        out = self.dropout2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out
        
cnn = CNN()
# cnn.cuda()

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn.parameters(), lr=learning_rate)

# Train the Model
for epoch in range(1):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images)#.cuda()
        labels = Variable(labels)#.cuda()
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = cnn(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [%d/%d], Iter [%d/%d] Loss: %.4f' 
                   %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))

# Test the Model
cnn.eval()    # Change model to 'eval' mode (BN uses moving mean/var).
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images).cuda()
    outputs = cnn(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted.cpu() == labels).sum()


Epoch [1/5], Iter [100/600] Loss: 0.5250
Epoch [1/5], Iter [200/600] Loss: 0.2036
Epoch [1/5], Iter [300/600] Loss: 0.2793
Epoch [1/5], Iter [400/600] Loss: 0.1463
Epoch [1/5], Iter [500/600] Loss: 0.1377
Epoch [1/5], Iter [600/600] Loss: 0.3383


AssertionError: Torch not compiled with CUDA enabled

In [6]:
images.size()

torch.Size([100, 1, 28, 28])

In [7]:
labels.size()

torch.Size([100])

In [62]:
a = Variable(torch.from_numpy(np.random.normal(size=[100, 1, 28, 28]))).cuda()

In [63]:
a

Variable containing:
( 0 , 0 ,.,.) = 
  7.6298e-01 -4.1437e-01 -9.0198e-01  ...   7.5805e-02 -1.1335e+00  6.5689e-01
 -6.7386e-01  2.3131e-01  2.5898e+00  ...   4.2166e-01 -4.3786e-01  4.2389e-01
 -7.1151e-01  1.0197e+00 -1.1271e+00  ...   1.5185e+00  7.4697e-01  7.9381e-01
                 ...                   ⋱                   ...                
  2.2206e+00 -1.1550e+00 -8.9005e-01  ...  -1.1206e+00 -4.9866e-02  2.9037e-01
 -5.6936e-01  4.9287e-01  1.1103e+00  ...  -7.8940e-01 -2.0891e+00  1.9518e-01
  6.7065e-01 -2.1191e+00  2.4680e-01  ...  -4.3550e-01  1.2272e-01 -1.1919e-01
      ⋮  

( 1 , 0 ,.,.) = 
  2.7667e-01  9.2498e-02  1.9687e-01  ...  -1.5553e+00 -9.7056e-01  1.3404e+00
  4.5240e-02 -2.6352e+00 -1.3836e+00  ...  -5.9137e-01  5.9493e-01 -3.7456e-01
  6.3916e-01 -1.3123e-01 -6.9574e-01  ...  -7.5272e-01  1.9242e-01  1.0774e+00
                 ...                   ⋱                   ...                
 -4.9260e-01 -5.8932e-01 -6.3558e-01  ...   6.7141e-01 -5.6990e-0